In [29]:
%run download-planet-data.py


In [34]:
# TESS API isn't working yet. Using wget.
#import pandas as pd
#NEXSCI_API = 'http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI'
#print('Downloading TESS confirmed planets from NExSci...')
#df = pd.read_csv(NEXSCI_API + '?table=exoplanets&select=*&where=pl_facility+like+%27TESS%27')
#df.to_csv('tess-confirmed-planets.csv')

In [51]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.io import push_notebook, show, output_notebook

# output to notebook
output_notebook()
# output to static HTML file
output_file('tess_example.html')

df = pd.read_csv('tess-confirmed-planets.csv')

# iterating the columns 
#for col in df.columns: 
#    print(col) 

 #df[['pl_orbper', 'pl_rade']]  
orb = df.loc[:, 'pl_orbper'] 
rad = df.loc[:, 'pl_rade'] 

#df[['pl_orbper', 'pl_rade']]  


Loading BokehJS ...

In [53]:

p.scatter(orb, rad)
show(p)